In [1]:
# 取通道、公区区域的客流数据

In [2]:
import sys 
sys.path.append('/home/aistudio/external-libraries/chinese_calendar')
sys.path.append('/home/aistudio/external-libraries/statsmodels')
sys.path.append('/home/aistudio/external-libraries/plotly')
sys.path.append('/home/aistudio/external-libraries/pmdarima')

In [3]:
import numpy as np
import pandas as pd
import pickle

from tqdm import tqdm
import math, random

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import date
from datetime import timedelta
import gc

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

import paddle
import paddle.nn as nn
from paddle.optimizer import AdamW

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

SEED = 2022

random.seed(SEED)
np.random.seed(SEED)
paddle.seed(SEED)

In [4]:
# 客流数据
area_custflow_hour = pd.read_csv(r"/home/aistudio/data/data141580/空间每小时客流数据.csv", encoding='utf-16')

## 客流数据预处理

In [5]:
# area_custflow_hour[area_custflow_hour.duplicated(keep=False)].sort_values('area_id')

In [6]:
#去重
area_custflow_hour.drop_duplicates(inplace=True)

In [7]:
# area_custflow_hour[(area_custflow_hour['area_id'] == 428943957888634880) * \
#                     (area_custflow_hour['create_date'] == 20220322) * (area_custflow_hour['stats_hour'] == 15)]

In [8]:
# area_custflow_hour.info()

In [9]:
area_custflow_hour['create_date'] = pd.to_datetime(area_custflow_hour['create_date'], format='%Y%m%d')

In [10]:
area_custflow_hour

,area_id,name,area_type,size,parent_id,path,attr_name_11,attr_name_12,attr_name_13,create_date,stats_hour,perno_out,perno_pre,stay_perno,perno_in,rate
0,428943957888634880,LG层客流区域,5,NaN,427492022861979648,828ef5167ba440c59d3e1a30c0bb594e|4274919930614...,NaN,楼层出入口,NaN,2021-08-01,9,616,0,-33,583,100.0
1,428943957888634880,LG层客流区域,5,NaN,427492022861979648,828ef5167ba440c59d3e1a30c0bb594e|4274919930614...,NaN,楼层出入口,NaN,2021-08-01,10,1659,0,870,2529,100.0
2,428943957888634880,LG层客流区域,5,NaN,427492022861979648,828ef5167ba440c59d3e1a30c0bb594e|4274919930614...,NaN,楼层出入口,NaN,2021-08-01,11,2796,0,797,3593,100.0
3,428943957888634880,LG层客流区域,5,NaN,427492022861979648,828ef5167ba440c59d3e1a30c0bb594e|4274919930614...,NaN,楼层出入口,NaN,2021-08-01,12,2835,0,839,3674,100.0
4,428943957888634880,LG层客流区域,5,NaN,427492022861979648,828ef5167ba440c59d3e1a30c0bb594e|4274919930614...,NaN,楼层出入口,NaN,2021-08-01,13,3305,0,887,4192,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251474,561308000296427520,光里入口4,5,NaN,429674366402318336,828ef5167ba440c59d3e1a30c0bb594e|4296412538695...,NaN,其他区域,NaN,2022-03-31,19,0,0,82,82,100.0
1251475,561308000296427520,光里入口4,5,NaN,429674366402318336,828ef5167ba440c59d3e1a30c0bb594e|4296412538695...,NaN,其他区域,NaN,2022-03-31,20,0,0,63,63,100.0
1251476,561308000296427520,光里入口4,5,NaN,429674366402318336,828ef5167ba440c59d3e1a30c0bb594e|4296412538695...,NaN,其他区域,NaN,2022-03-31,21,0,0,35,35,100.0
1251477,561308000296427520,光里入口4,5,NaN,429674366402318336,828ef5167ba440c59d3e1a30c0bb594e|4296412538695...,NaN,其他区域,NaN,2022-03-31,22,0,0,17,17,100.0


In [11]:
# area_list = ['区域', '入口', '出口', '扶梯', '升降梯', '观光梯', '接驳口', '通往',]

In [12]:
# tmp_area_custflow_hour = pd.DataFrame()
# for area_id in area_custflow_hour['area_id'].unique():
#     name = area_custflow_hour[area_custflow_hour['area_id'] == area_id]['name'].unique()[0]
#     if any([(n in name) for n in area_list]):
#         tmp_area_custflow_hour = tmp_area_custflow_hour.append(area_custflow_hour[area_custflow_hour['area_id'] == area_id])

In [13]:
# tmp_area_custflow_hour

In [14]:
tmp_area_custflow_hour = area_custflow_hour.pivot(index=['create_date', 'stats_hour'],
                                                  columns=['area_id'],
                                                  values=['perno_out', 'perno_pre', 'stay_perno', \
                                                          'perno_in'])

In [15]:
tmp_area_custflow_hour.iloc[[-2]]#.loc['2022-03-29', :]

perno_out                     \
area_id                428943957888634880 429311602932600832   
create_date stats_hour                                         
2022-03-31  22                      381.0              287.0   

                                                              \
area_id                429312369911418880 429312498865295360   
create_date stats_hour                                         
2022-03-31  22                      113.0               94.0   

                                                              \
area_id                429313014387200000 429313082943098880   
create_date stats_hour                                         
2022-03-31  22                      134.0               69.0   

                                                              \
area_id                429703355472818176 429703649367699456   
create_date stats_hour                                         
2022-03-31  22                        7.0                1.0   

                                                              ...  \
area_id                429703714119364608 429703762857177088  ...   
create_date stats_hour                                        ...   
2022-03-31  22                        NaN               74.0  ...   

                                 perno_in                     \
area_id                528907625501306880 528907662142775296   
create_date stats_hour                                         
2022-03-31  22                       13.0               23.0   

                                                              \
area_id                528907783563649024 528907788231962624   
create_date stats_hour                                         
2022-03-31  22                       15.0                4.0   

                                                              \
area_id                528907837217234944 560930937903312896   
create_date stats_hour                                         
2022-03-31  22                       53.0              192.0   

                                                              \
area_id                561307819966480384 561307869710921728   
create_date stats_hour                                         
2022-03-31  22                      134.0               31.0   

                                                              
area_id                561307948500918272 561308000296427520  
create_date stats_hour                                        
2022-03-31  22                       10.0               17.0  

[1 rows x 604 columns]

In [16]:
tmp_area_custflow_hour = tmp_area_custflow_hour.fillna(method='bfill')
# tmp_area_custflow_hour.fillna(0.0001, inplace=True)
tmp_area_custflow_hour.dropna(axis=1, inplace=True)

In [17]:
tmp_area_custflow_hour

perno_out                     \
area_id                428943957888634880 429311602932600832   
create_date stats_hour                                         
2021-08-01  9                       616.0              236.0   
            10                     1659.0             1410.0   
            11                     2796.0             2332.0   
            12                     2835.0             2294.0   
            13                     3305.0             2555.0   
...                                   ...                ...   
2022-03-31  19                     2216.0             1454.0   
            20                     1914.0             1379.0   
            21                     1200.0              798.0   
            22                      381.0              287.0   
            23                       62.0               78.0   

                                                              \
area_id                429312369911418880 429312498865295360   
create_date stats_hour                                         
2021-08-01  9                       136.0               86.0   
            10                      581.0              488.0   
            11                     1166.0             1538.0   
            12                     1194.0             1816.0   
            13                     1347.0             1882.0   
...                                   ...                ...   
2022-03-31  19                      668.0              859.0   
            20                      770.0              741.0   
            21                      394.0              393.0   
            22                      113.0               94.0   
            23                       19.0               31.0   

                                                              \
area_id                429313014387200000 429313082943098880   
create_date stats_hour                                         
2021-08-01  9                       127.0               79.0   
            10                      243.0              209.0   
            11                      753.0              357.0   
            12                     1376.0              760.0   
            13                     1988.0             1190.0   
...                                   ...                ...   
2022-03-31  19                      834.0              381.0   
            20                      757.0              351.0   
            21                      475.0              198.0   
            22                      134.0               69.0   
            23                       64.0               41.0   

                                                              \
area_id                429703762857177088 429703816233889792   
create_date stats_hour                                         
2021-08-01  9                        66.0               38.0   
            10                       66.0               69.0   
            11                       66.0              111.0   
            12                       66.0              186.0   
            13                       66.0              258.0   
...                                   ...                ...   
2022-03-31  19                      338.0              135.0   
            20                      302.0               96.0   
            21                      246.0               78.0   
            22                       74.0               15.0   
            23                       34.0                4.0   

                                                              ...  \
area_id                429703950753607680 429703998157631488  ...   
create_date stats_hour                                        ...   
2021-08-01  9                        27.0                4.0  ...   
            10                       49.0              191.0  ...   
            11                      111.0              244.0  ...   
            12                

In [18]:
tmp_area_custflow_hour = tmp_area_custflow_hour.stack().reset_index()\
                                               .groupby(by=['create_date', 'area_id'])\
                                               ['perno_out', 'perno_pre', 'stay_perno', 'perno_in']\
                                               .agg(['sum', 'mean'])

In [19]:
tmp_area_custflow_hour

perno_out              perno_pre       \
                                     sum         mean       sum mean   
create_date area_id                                                    
2021-08-01  428943957888634880   43261.0  2884.066667       0.0  0.0   
            429311602932600832   34564.0  2304.266667       0.0  0.0   
            429312369911418880   17048.0  1136.533333       0.0  0.0   
            429312498865295360   21933.0  1462.200000       0.0  0.0   
            429313014387200000   15465.0  1031.000000       0.0  0.0   
...                                  ...          ...       ...  ...   
2022-03-31  560930937903312896       0.0     0.000000       0.0  0.0   
            561307819966480384       0.0     0.000000       0.0  0.0   
            561307869710921728       0.0     0.000000       0.0  0.0   
            561307948500918272       0.0     0.000000       0.0  0.0   
            561308000296427520       0.0     0.000000       0.0  0.0   

                               stay_perno             perno_in               
                                      sum        mean      sum         mean  
create_date area_id                                                          
2021-08-01  428943957888634880     9211.0  614.066667  52472.0  3498.133333  
            429311602932600832    -3084.0 -205.600000  31480.0  2098.666667  
            429312369911418880     3810.0  254.000000  20858.0  1390.533333  
            429312498865295360     1141.0   76.066667  23074.0  1538.266667  
            429313014387200000      468.0   31.200000  15933.0  1062.200000  
...                                   ...         ...      ...          ...  
2022-03-31  560930937903312896     5287.0  330.437500   5287.0   330.437500  
            561307819966480384     2102.0  131.375000   2102.0   131.375000  
            561307869710921728     1826.0  114.125000   1826.0   114.125000  
            561307948500918272      406.0   25.375000    406.0    25.375000  
            561308000296427520      953.0   59.562500    953.0    59.562500  

[25029 rows x 8 columns]

In [20]:
cols = []
for col in tmp_area_custflow_hour.columns:
    cols.append(col[0] + '_' + col[1])
tmp_area_custflow_hour.columns = cols

In [21]:
val_cols = tmp_area_custflow_hour.columns
tmp_area_custflow_hour = tmp_area_custflow_hour.reset_index().pivot(index=['create_date'],
                                                                    columns=['area_id'],
                                                                    values=val_cols)

In [22]:
df_area_custflow = tmp_area_custflow_hour.T

In [23]:
df_area_custflow

create_date                       2021-08-01  2021-08-02  2021-08-03  \
              area_id                                                  
perno_out_sum 428943957888634880     43261.0     26130.0     26089.0   
              429311602932600832     34564.0     20085.0     20395.0   
              429312369911418880     17048.0      9262.0      9480.0   
              429312498865295360     21933.0     11725.0     11885.0   
              429313014387200000     15465.0      8827.0      8975.0   
...                                      ...         ...         ...   
perno_in_mean 560930937903312896       399.0       399.0       399.0   
              561307819966480384         1.0         1.0         1.0   
              561307869710921728         1.0         1.0         1.0   
              561307948500918272         1.0         1.0         1.0   
              561308000296427520         1.0         1.0         1.0   

create_date                       2021-08-04  2021-08-05  2021-08-06  \
              area_id                                                  
perno_out_sum 428943957888634880     26862.0     25449.0     28669.0   
              429311602932600832     20741.0     18428.0     19621.0   
              429312369911418880      9946.0      8375.0      9495.0   
              429312498865295360     12158.0     10940.0     12264.0   
              429313014387200000      9561.0      8927.0      9781.0   
...                                      ...         ...         ...   
perno_in_mean 560930937903312896       399.0       399.0       399.0   
              561307819966480384         1.0         1.0         1.0   
              561307869710921728         1.0         1.0         1.0   
              561307948500918272         1.0         1.0         1.0   
              561308000296427520         1.0         1.0         1.0   

create_date                       2021-08-07  2021-08-08  2021-08-09  \
              area_id                                                  
perno_out_sum 428943957888634880     48102.0     47444.0     31614.0   
              429311602932600832     34924.0     34234.0     22584.0   
              429312369911418880     18112.0     18170.0     10693.0   
              429312498865295360     23674.0     23823.0     13486.0   
              429313014387200000     16041.0     16521.0      9754.0   
...                                      ...         ...         ...   
perno_in_mean 560930937903312896       399.0       399.0       399.0   
              561307819966480384         1.0         1.0         1.0   
              561307869710921728         1.0         1.0         1.0   
              561307948500918272         1.0         1.0         1.0   
              561308000296427520         1.0         1.0         1.0   

create_date                       2021-08-10  ...  2022-03-22  2022-03-23  \
              area_id                         ...                           
perno_out_sum 428943957888634880     30310.0  ...     16320.0     21160.0   
              429311602932600832     22332.0  ...     11187.0     14198.0   
              429312369911418880     10340.0  ...      5223.0      6731.0   
              429312498865295360     13769.0  ...      4693.0      5853.0   
              429313014387200000      9805.0  ...      4283.0      5215.0   
...                                      ...  ...         ...         ...   
perno_in_mean 560930937903312896       399.0  ...       399.0       399.0   
              561307819966480384         1.0  ...         1.0         1.0   
              561307869710921728         1.0  ...         1.0         1.0   
              561307948500918272         1.0  ...         1.0         1.0   
              561308000296427520         1.0  ...         1.0         1.0   

create_date                       2022-03-24  2022-03-25  2022-03-26  \
              area_id                                                  
perno_out_sum 428943957888634880     18613.

In [24]:
# 导出数据
df_area_custflow.to_pickle('./work/AI_data/df_area_custflow.pkl')